# Augur for Influenza A - HA protein

In [1]:
import json

## Metadata is required

Making metadata by parsing FASTA

In [278]:
from Bio import SeqIO
import re


records = []
for record in SeqIO.parse("../data/fluHA_sequences.fasta", "fasta"):
    records.append(record.description)

# Can I compile these regexes together? 

# Get strains
strainsearch = re.compile('Strain Name:(?P<strain>.*\d{2}|d{4})')

# Get ids
idsearch = re.compile('gb:(?P<id>[A-Z]+\d+[:]\d+[-]\d{4})')
# idsearch_exact = re.compile('(?P<id>gb:[A-Z]+\d+[:]\d+[-]\d{4})')

strainsList = []
idList = []
for record in records:
        i = idsearch.search(record)
#         i = idsearch_exact.search(record)
        id = i.group('id')
        idList.append(id)

        s = strainsearch.search(record)
        strain = s.group('strain')
        strainsList.append(strain)


# Get years
yearsSearch = re.compile('(?P<year>\d{4}$|\d{2}$)')
yearsList = []
for strain in strainsList:
    y = yearsSearch.search(strain)
    year = y.group('year')
    yearsList.append(year)

# Concatenate strain strings with id strings for unique identifiers
newID = []
counter = 0
for id in idList:
    newID.append(id + "|" + strainsList[counter])
    
    # Remove spaces
    if " " in newID[counter]:
        newID[counter] = newID[counter].replace(" ", "")
    counter += 1

# for id in newID:
#     print(id)
# metadata has columns:
# strain	virus	accession	date	region	country	division	city	db	segment


J02176:1-1775|A/WSN/1933
CY021709:19-1719|A/AA/Huston/1945
CY020285:11-1711|A/AA/Marton/1943
CY025381:16-1713|A/Alabama/UR06-0455/2007
CY027411:1-1698|A/Alabama/UR06-0536/2007
EU716540:1-1698|A/Alaska/02/2008
EU716555:1-1698|A/Alaska/03/2008
FJ687001:1-1698|A/Alaska/05/2008
EU516254:1-1698|A/Alaska/11/2007
EU516256:1-1698|A/Alaska/13/2007
CY019955:11-1708|A/Alaska/1935
CY021821:17-1717|A/Albany/12/1951
CY022093:18-1718|A/Albany/13/1951
CY022021:17-1717|A/Albany/14/1951
CY021901:17-1717|A/Albany/1618/1951
CY021797:17-1717|A/Albany/20/1978
CY019947:11-1711|A/Albany/4835/1948
CY021701:12-1712|A/Albany/4836/1950
CY026411:1-1701|A/Albany/8/1979
S62154:33-1733|A/AlmaAta/1417/84
EU516243:1-1698|A/Arizona/03/2007
EU566991:1-1698|A/Arizona/04/2007
EU516246:1-1698|A/Arizona/07/2007
FJ179358:1-1698|A/Arizona/13/2007
FJ686985:1-1698|A/Arizona/13/2008
FJ686949:1-1698|A/Arizona/13/2008
CY019963:11-1711|A/Arizona/14/1978
FJ179359:1-1698|A/Arizona/14/2007
EU566966:1-1698|A/Arizona/15/2007
EU199351:1-1

In [279]:
from Bio import SeqIO

original_file = r"../data/fluHA_sequences.fasta"
corrected_file = r"../data/corrected.fasta"

position = 0
with open(original_file) as original, open(corrected_file, 'w') as corrected:
    records = SeqIO.parse(original_file, 'fasta')
    for record in records:
#         print(record.id)
        record.id = newID[position]
        record.description = newID[position]
        position += 1
#         print(record.id)             
        SeqIO.write(record, corrected, 'fasta')

gb:J02176:1-1775|Organism:Influenza
J02176:1-1775|A/WSN/1933
gb:CY021709:19-1719|Organism:Influenza
CY021709:19-1719|A/AA/Huston/1945
gb:CY020285:11-1711|Organism:Influenza
CY020285:11-1711|A/AA/Marton/1943
gb:CY025381:16-1713|Organism:Influenza
CY025381:16-1713|A/Alabama/UR06-0455/2007
gb:CY027411:1-1698|Organism:Influenza
CY027411:1-1698|A/Alabama/UR06-0536/2007
gb:EU716540:1-1698|Organism:Influenza
EU716540:1-1698|A/Alaska/02/2008
gb:EU716555:1-1698|Organism:Influenza
EU716555:1-1698|A/Alaska/03/2008
gb:FJ687001:1-1698|Organism:Influenza
FJ687001:1-1698|A/Alaska/05/2008
gb:EU516254:1-1698|Organism:Influenza
EU516254:1-1698|A/Alaska/11/2007
gb:EU516256:1-1698|Organism:Influenza
EU516256:1-1698|A/Alaska/13/2007
gb:CY019955:11-1708|Organism:Influenza
CY019955:11-1708|A/Alaska/1935
gb:CY021821:17-1717|Organism:Influenza
CY021821:17-1717|A/Albany/12/1951
gb:CY022093:18-1718|Organism:Influenza
CY022093:18-1718|A/Albany/13/1951
gb:CY022021:17-1717|Organism:Influenza
CY022021:17-1717|A/Alba

gb:CY020445:11-1711|Organism:Influenza
CY020445:11-1711|A/Henry/1936
gb:FJ231766:1-1698|Organism:Influenza
FJ231766:1-1698|A/Hessen/4/2005
gb:CY013271:11-1711|Organism:Influenza
CY013271:11-1711|A/Hickox/1940
gb:CY146777:11-1711|Organism:Influenza
CY146777:11-1711|A/Hickox/JY2/1940
gb:DQ397950:1-1698|Organism:Influenza
DQ397950:1-1698|A/Hiroshima/37/2001
gb:CY105158:18-1715|Organism:Influenza
CY105158:18-1715|A/HoChiMinh/HCM395/2006
gb:CY105166:18-1715|Organism:Influenza
CY105166:18-1715|A/HoChiMinh/HCM526/2006
gb:CY104662:18-1715|Organism:Influenza
CY104662:18-1715|A/HoChiMinh/HCM554/2006
gb:CY105174:18-1715|Organism:Influenza
CY105174:18-1715|A/HoChiMinh/HCM639/2006
gb:CY043425:33-1730|Organism:Influenza
CY043425:33-1730|A/Hokkaido/07H007/2007
gb:AF386778:33-1730|Organism:Influenza
AF386778:33-1730|A/HongKong/1035/98
gb:AF386777:33-1730|Organism:Influenza
AF386777:33-1730|A/HongKong/1035/98
gb:AF386776:33-1730|Organism:Influenza
AF386776:33-1730|A/HongKong/1035/98
gb:AF386775:33-1730

gb:FJ743462:31-1728|Organism:Influenza
FJ743462:31-1728|A/NewCaledonia/6/2008
gb:GQ466359:1-1698|Organism:Influenza
GQ466359:1-1698|A/NewJersey/01/2008
gb:EU566986:1-1698|Organism:Influenza
EU566986:1-1698|A/NewJersey/02/2008
gb:EU566993:1-1698|Organism:Influenza
EU566993:1-1698|A/NewJersey/03/2008
gb:EU567007:1-1698|Organism:Influenza
EU567007:1-1698|A/NewJersey/04/2008
gb:EU567013:1-1698|Organism:Influenza
EU567013:1-1698|A/NewJersey/05/2008
gb:EU566979:1-1698|Organism:Influenza
EU566979:1-1698|A/NewJersey/06/2008
gb:GQ475687:1-1698|Organism:Influenza
GQ475687:1-1698|A/NewJersey/06/2008
gb:EU716548:1-1698|Organism:Influenza
EU716548:1-1698|A/NewJersey/08/2008
gb:EU516143:1-1698|Organism:Influenza
EU516143:1-1698|A/NewJersey/09/2007
gb:EU716619:1-1698|Organism:Influenza
EU716619:1-1698|A/NewJersey/09/2008
gb:EU779618:1-1698|Organism:Influenza
EU779618:1-1698|A/NewJersey/10/2008
gb:CY044365:1-1701|Organism:Influenza
CY044365:1-1701|A/NewJersey/11/1976
gb:EU887032:1-1698|Organism:Influe

gb:KF533064:1-1698|Organism:Influenza
KF533064:1-1698|A/Singapore/106L/2007
gb:KF533062:1-1698|Organism:Influenza
KF533062:1-1698|A/Singapore/115C/2007
gb:CY125148:17-1714|Organism:Influenza
CY125148:17-1714|A/Singapore/14/2001
gb:KF856946:1-1698|Organism:Influenza
KF856946:1-1698|A/Singapore/20J/2007
gb:KF856947:1-1698|Organism:Influenza
KF856947:1-1698|A/Singapore/23J/2007
gb:KP223140:1-1698|Organism:Influenza
KP223140:1-1698|A/Singapore/24D/2007
gb:KP223141:1-1698|Organism:Influenza
KP223141:1-1698|A/Singapore/30L/2007
gb:KP223142:1-1698|Organism:Influenza
KP223142:1-1698|A/Singapore/44T/2007
gb:KF533063:1-1698|Organism:Influenza
KF533063:1-1698|A/Singapore/49O/2007
gb:CY020477:11-1711|Organism:Influenza
CY020477:11-1711|A/Singapore/6/1986
gb:KP223143:1-1698|Organism:Influenza
KP223143:1-1698|A/Singapore/86D/2007
gb:KP223144:1-1698|Organism:Influenza
KP223144:1-1698|A/Singapore/89Z/2007
gb:EU100724:1-1698|Organism:Influenza
EU100724:1-1698|A/SolomonIslands/03/2006
gb:EU124177:33-173

In [280]:
import pandas as pd

# metadata_id = list(zip(fasta_ids, strainsList, yearsList))
# metadata = list(zip(idList, strainsList, yearsList))
metadata = list(zip(newID, yearsList))

# df = pd.DataFrame(data = metadata, columns=['id', 'strain', 'year'])
df = pd.DataFrame(data = metadata, columns=['strain', 'year'])

df.to_csv('../data/metadata.tsv',sep='\t',index=False,header=True)

                                           strain  year
0                        J02176:1-1775|A/WSN/1933  1933
1               CY021709:19-1719|A/AA/Huston/1945  1945
2               CY020285:11-1711|A/AA/Marton/1943  1943
3       CY025381:16-1713|A/Alabama/UR06-0455/2007  2007
4        CY027411:1-1698|A/Alabama/UR06-0536/2007  2007
5                EU716540:1-1698|A/Alaska/02/2008  2008
6                EU716555:1-1698|A/Alaska/03/2008  2008
7                FJ687001:1-1698|A/Alaska/05/2008  2008
8                EU516254:1-1698|A/Alaska/11/2007  2007
9                EU516256:1-1698|A/Alaska/13/2007  2007
10                 CY019955:11-1708|A/Alaska/1935  1935
11              CY021821:17-1717|A/Albany/12/1951  1951
12              CY022093:18-1718|A/Albany/13/1951  1951
13              CY022021:17-1717|A/Albany/14/1951  1951
14            CY021901:17-1717|A/Albany/1618/1951  1951
15              CY021797:17-1717|A/Albany/20/1978  1978
16            CY019947:11-1711|A/Albany/4835/194

In [282]:
!mkdir -p ../results/

!augur filter \
  --sequences ../data/corrected.fasta \
  --metadata ../data/metadata.tsv \
  #--exclude zika-tutorial/config/dropped_strains.txt \
  --output ../results/filtered.fasta \
  --group-by year \
  --sequences-per-group 2 \
#   --min-date 1918

Traceback (most recent call last):
  File "/home/gboyle/.local/bin//augur", line 11, in <module>
    sys.exit(main())
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__main__.py", line 10, in main
    return augur.run( argv[1:] )
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__init__.py", line 66, in run
    return args.__command__.run(args)
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/filter.py", line 293, in run
    SeqIO.write(seq_to_keep, args.output, 'fasta')
  File "/home/gboyle/.local/lib/python3.6/site-packages/Bio/SeqIO/__init__.py", line 529, in write
    fp.write(format_function(record))
AttributeError: 'NoneType' object has no attribute 'write'


### Does pairing down the zika metadata result in the same error?

In [289]:
zika = pd.read_csv("../data/metadata-Copy1.tsv", sep='\t')

col_list = ['strain','date']
newmeta = zika[col_list]
newmeta.to_csv('../data/new-zika-metadata.tsv',sep='\t',index=False,header=True)


                         strain        date
0                     1_0087_PF  2013-12-XX
1                     1_0181_PF  2013-12-XX
2                     1_0199_PF  2013-11-XX
3   Aedes_aegypti/USA/2016/FL05  2016-09-09
4              BRA/2016/FC_6706  2016-04-08
5           Brazil/2015/ZBRA105  2015-02-23
6           Brazil/2015/ZBRC301  2015-05-13
7           Brazil/2015/ZBRC303  2015-05-14
8            Brazil/2016/ZBRC16  2016-01-19
9            COL/FLR_00008/2015  2015-12-XX
10           COL/FLR_00024/2015  2015-12-XX
11           COL/PRV_00028/2015  2016-12-XX
12        Colombia/2016/ZC204Se  2016-01-06
13             DOM/2016/BB_0059  2016-04-04
14             DOM/2016/BB_0183  2016-04-18
15             DOM/2016/BB_0433  2016-06-13
16        DOM/2016/MA_WGS16_011  2016-06-06
17                   EcEs062_16  2016-04-XX
18             HND/2016/HU_ME59  2016-05-13
19                     Nica1_16  2016-01-19
20    PAN/CDC_259359_V1_V3/2015  2015-12-18
21                     PRVABC59 

In [294]:
!mkdir -p ../results/

!augur filter \
  --sequences ../data/sequences-Copy1.fasta \
  --metadata ../data/new-zika-metadata.tsv \
  #--exclude zika-tutorial/config/dropped_strains.txt \
  --output ../results/filtered.fasta \
  --group-by year \
  --sequences-per-group 2 \
#   --min-date 1918

Traceback (most recent call last):
  File "/home/gboyle/.local/bin//augur", line 11, in <module>
    sys.exit(main())
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__main__.py", line 10, in main
    return augur.run( argv[1:] )
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/__init__.py", line 66, in run
    return args.__command__.run(args)
  File "/home/gboyle/.local/lib/python3.6/site-packages/augur/filter.py", line 293, in run
    SeqIO.write(seq_to_keep, args.output, 'fasta')
  File "/home/gboyle/.local/lib/python3.6/site-packages/Bio/SeqIO/__init__.py", line 529, in write
    fp.write(format_function(record))
AttributeError: 'NoneType' object has no attribute 'write'


### How much of the metadata can we remove and still have it survive the `augur filter` command?